In [ ]:
import gzip
import json
import pandas as pd
import ast
from ast import literal_eval
import dateutil.parser as dparser
import re
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [ ]:
# Descargar los recursos necesarios (si no los has descargado previamente)
nltk.download('vader_lexicon')

# Crear una instancia del analizador de sentimientos
sia = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


In [ ]:
def abrir_comprimido(archivo):
    data_list = []
    with gzip.open(archivo, 'rb') as file:
        for line in file:
            decode_line = line.decode('utf-8')
            data_list.append(ast.literal_eval(decode_line.strip()))
        return data_list

In [ ]:
archivo = '/content/drive/My Drive/Steam_Data/user_reviews.json.gz'
lista_reviews = abrir_comprimido(archivo)
df_reviews = pd.DataFrame(lista_reviews)

In [ ]:
df_reviews

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."
...,...,...,...
25794,76561198306599751,http://steamcommunity.com/profiles/76561198306...,"[{'funny': '', 'posted': 'Posted May 31.', 'la..."
25795,Ghoustik,http://steamcommunity.com/id/Ghoustik,"[{'funny': '', 'posted': 'Posted June 17.', 'l..."
25796,76561198310819422,http://steamcommunity.com/profiles/76561198310...,"[{'funny': '1 person found this review funny',..."
25797,76561198312638244,http://steamcommunity.com/profiles/76561198312...,"[{'funny': '', 'posted': 'Posted July 21.', 'l..."


In [ ]:
archivo_csv = '/content/drive/My Drive/DataOutput_mvp_steam/user_reviews.csv'
df_reviews.to_csv(archivo_csv, index=False)

In [ ]:
user_reviews = pd.read_csv('/content/drive/My Drive/DataOutput_mvp_steam/user_reviews.csv')

In [ ]:
user_reviews

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."
...,...,...,...
25794,76561198306599751,http://steamcommunity.com/profiles/76561198306...,"[{'funny': '', 'posted': 'Posted May 31.', 'la..."
25795,Ghoustik,http://steamcommunity.com/id/Ghoustik,"[{'funny': '', 'posted': 'Posted June 17.', 'l..."
25796,76561198310819422,http://steamcommunity.com/profiles/76561198310...,"[{'funny': '1 person found this review funny',..."
25797,76561198312638244,http://steamcommunity.com/profiles/76561198312...,"[{'funny': '', 'posted': 'Posted July 21.', 'l..."


In [ ]:
user_reviews['reviews'] = user_reviews['reviews'].apply(ast.literal_eval)

In [ ]:
# Función para extraer el número y convertirlo a entero
def extract_and_convert_funny(review_list):
    for review in review_list:
        if 'funny' in review:
            # Extraer el número del texto utilizando una expresión regular
            match = re.search(r'\d+', review['funny'])
            if match:
                # Convertir el número a entero y reemplazar el valor en el diccionario
                review['funny'] = int(match.group())
    return review_list

In [ ]:
# Aplicar la función a la columna 'reviews'
user_reviews['reviews'] = user_reviews['reviews'].apply(extract_and_convert_funny)

In [ ]:
string = 'Posted May 5.'
tifg = dparser.parse(string, fuzzy=True)

In [ ]:
tifg

datetime.datetime(2023, 5, 5, 0, 0)

In [ ]:
tifg.strftime("%Y/%m/%d")

'2023/05/05'

In [ ]:
def extract_and_convert_dates(review_list, clave):
    for review in review_list:
        if clave in review:
            date_text = review[clave]
            try:
                # Intenta convertir la fecha utilizando dateutil.parser
                date_obj = dparser.parse(date_text, fuzzy=True)
                review[clave] = date_obj.strftime("%Y/%m/%d")
            except ValueError:
                pass
    return review_list

In [ ]:
# Aplicar la función a la clave 'posted'
user_reviews['reviews'] = user_reviews['reviews'].apply(lambda x: extract_and_convert_dates(x, clave='posted'))

In [ ]:
# Aplicar la función a la clave 'last_edited'
user_reviews['reviews'] = user_reviews['reviews'].apply(lambda x: extract_and_convert_dates(x, clave='last_edited'))

In [ ]:
# Función para convertir 'item_id' a entero
def convert_item_id_to_int(review_list):
    for review in review_list:
        if 'item_id' in review:
            try:
                # Intenta convertir 'item_id' a entero y actualizar el valor en el diccionario
                review['item_id'] = int(review['item_id'])
            except ValueError:
                pass
    return review_list

In [ ]:
# Aplicar la función a la clave 'item_id' de cada fila
user_reviews['reviews'] = user_reviews['reviews'].apply(convert_item_id_to_int)

In [ ]:
def convert_helpful_to_list(review_list):
    for review in review_list:
        if 'helpful' in review:
            helpful_str = review['helpful']
            numbers = re.findall(r'(\d+)\s*of\s*(\d+)\s*people', helpful_str, flags=re.IGNORECASE)
            if numbers:
                first_number, second_number = map(int, numbers[0])
                review['helpful'] = [first_number, second_number]
    return review_list

In [ ]:
user_reviews['reviews'] = user_reviews['reviews'].apply(convert_helpful_to_list)

In [ ]:
(user_reviews['reviews'][654][3]['helpful'])

'No ratings yet'

In [ ]:
user_reviews['reviews'][654]

[{'funny': 2,
  'posted': '2015/05/18',
  'last_edited': '',
  'item_id': 227680,
  'helpful': [2, 3],
  'recommend': False,
  'review': '♥♥♥♥ ass game all round'},
 {'funny': '',
  'posted': '2023/03/29',
  'last_edited': '',
  'item_id': 251570,
  'helpful': [1, 3],
  'recommend': True,
  'review': 'keep thoose updates flowing game is becomming a muth♥♥♥♥ing beast'},
 {'funny': '',
  'posted': '2015/06/06',
  'last_edited': '',
  'item_id': 264710,
  'helpful': 'No ratings yet',
  'recommend': True,
  'review': 'awsome game nothing else like it'},
 {'funny': '',
  'posted': '2014/06/19',
  'last_edited': '2015/10/17',
  'item_id': 263440,
  'helpful': 'No ratings yet',
  'recommend': False,
  'review': '♥♥♥♥ing ♥♥♥♥ house game'}]

In [ ]:
user_reviews

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': '2011/11/05', 'last_e..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': '2014/06/24', 'last_e..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': '2023/02/03', 'last_e..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': '2013/10/14', 'last_e..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': 3, 'posted': '2014/04/15', 'last_ed..."
...,...,...,...
25794,76561198306599751,http://steamcommunity.com/profiles/76561198306...,"[{'funny': '', 'posted': '2023/05/31', 'last_e..."
25795,Ghoustik,http://steamcommunity.com/id/Ghoustik,"[{'funny': '', 'posted': '2023/06/17', 'last_e..."
25796,76561198310819422,http://steamcommunity.com/profiles/76561198310...,"[{'funny': 1, 'posted': '2023/06/23', 'last_ed..."
25797,76561198312638244,http://steamcommunity.com/profiles/76561198312...,"[{'funny': '', 'posted': '2023/07/21', 'last_e..."


In [ ]:
user_reviews[user_reviews['user_id'].isnull()]

,user_id,user_url,reviews


In [ ]:
len(user_reviews[user_reviews['user_id'].duplicated(keep=False)])

623

In [ ]:
user_reviews = user_reviews.drop_duplicates(subset='user_id')

In [ ]:
user_reviews[user_reviews['user_url'].isnull()]

,user_id,user_url,reviews


In [ ]:
len(user_reviews[user_reviews['user_url'].duplicated(keep=False)])

0

In [ ]:
user_reviews = user_reviews.reset_index(drop=True)

In [ ]:
user_reviews['reviews'][0]

[{'funny': '',
  'posted': '2011/11/05',
  'last_edited': '',
  'item_id': 1250,
  'helpful': 'No ratings yet',
  'recommend': True,
  'review': 'Simple yet with great replayability. In my opinion does "zombie" hordes and team work better than left 4 dead plus has a global leveling system. Alot of down to earth "zombie" splattering fun for the whole family. Amazed this sort of FPS is so rare.'},
 {'funny': '',
  'posted': '2011/07/15',
  'last_edited': '',
  'item_id': 22200,
  'helpful': 'No ratings yet',
  'recommend': True,
  'review': "It's unique and worth a playthrough."},
 {'funny': '',
  'posted': '2011/04/21',
  'last_edited': '',
  'item_id': 43110,
  'helpful': 'No ratings yet',
  'recommend': True,
  'review': 'Great atmosphere. The gunplay can be a bit chunky at times but at the end of the day this game is definitely worth it and I hope they do a sequel...so buy the game so I get a sequel!'}]

In [ ]:
user_reviews

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': '2011/11/05', 'last_e..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': '2014/06/24', 'last_e..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': '2023/02/03', 'last_e..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': '2013/10/14', 'last_e..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': 3, 'posted': '2014/04/15', 'last_ed..."
...,...,...,...
25480,76561198306599751,http://steamcommunity.com/profiles/76561198306...,"[{'funny': '', 'posted': '2023/05/31', 'last_e..."
25481,Ghoustik,http://steamcommunity.com/id/Ghoustik,"[{'funny': '', 'posted': '2023/06/17', 'last_e..."
25482,76561198310819422,http://steamcommunity.com/profiles/76561198310...,"[{'funny': 1, 'posted': '2023/06/23', 'last_ed..."
25483,76561198312638244,http://steamcommunity.com/profiles/76561198312...,"[{'funny': '', 'posted': '2023/07/21', 'last_e..."


      Feature Engineering: Análisis de sentimientos

In [ ]:
# Función para asignar un valor numérico a los comentarios según su sentimiento
def asignar_valor_sentimiento(comentario, margen=0.05):
    sentiment = sia.polarity_scores(comentario['review'])['compound']
    if sentiment < -margen:  # Si es negativo
        comentario['review'] = 0
    elif sentiment > margen:  # Si es positivo
        comentario['review'] = 2
    else:  # Si es neutro
        comentario['review'] = 1
    return comentario

# Aplicar la función a cada fila del DataFrame
user_reviews['reviews'] = user_reviews['reviews'].apply(lambda comentarios: [asignar_valor_sentimiento(comentario) for comentario in comentarios])

In [ ]:
user_reviews

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': '2011/11/05', 'last_e..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': '2014/06/24', 'last_e..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': '2023/02/03', 'last_e..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': '2013/10/14', 'last_e..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': 3, 'posted': '2014/04/15', 'last_ed..."
...,...,...,...
25480,76561198306599751,http://steamcommunity.com/profiles/76561198306...,"[{'funny': '', 'posted': '2023/05/31', 'last_e..."
25481,Ghoustik,http://steamcommunity.com/id/Ghoustik,"[{'funny': '', 'posted': '2023/06/17', 'last_e..."
25482,76561198310819422,http://steamcommunity.com/profiles/76561198310...,"[{'funny': 1, 'posted': '2023/06/23', 'last_ed..."
25483,76561198312638244,http://steamcommunity.com/profiles/76561198312...,"[{'funny': '', 'posted': '2023/07/21', 'last_e..."


      Crear df_para_steam_games

In [ ]:
# Lista para almacenar los datos de cada review
datos_reviews = []

# Iterar sobre las filas del DataFrame
for index, row in user_reviews.iterrows():
    # Iterar sobre las reviews en la columna 'reviews'
    for review in row['reviews']:
        item_id = review['item_id']
        recommend = review['recommend']
        review_value = review['review']

        # Agregar los datos de la review a la lista
        datos_reviews.append([item_id, recommend, review_value])

# Crear un DataFrame a partir de la lista de datos
df = pd.DataFrame(datos_reviews, columns=['item_id', 'recommend', 'review'])

# Convertir la columna 'recommend' en números (True=1, False=0)
df['recommend'] = df['recommend'].astype(int)

# Agrupar por 'item_id' y aplicar las operaciones de agregación
agregacion = {
    'recommend': ['sum', 'count'],
    'review': list
}

df_para_steam_games = df.groupby('item_id').agg(agregacion).reset_index()

# Renombrar las columnas 'recommend' y 'review'
df_para_steam_games.columns = ['item_id', 'recommend_sum', 'recommend_count', 'reviews']

# Crear la columna 'recommend' como una lista [False, True]
df_para_steam_games['recommend'] = df_para_steam_games.apply(lambda row: [row['recommend_count'] - row['recommend_sum'], row['recommend_count']], axis=1)


In [ ]:
df_para_steam_games

,item_id,recommend_sum,recommend_count,reviews,recommend
0,10,55,56,"[2, 1, 2, 2, 2, 1, 2, 1, 2, 2, 2, 2, 1, 2, 2, ...","[1, 56]"
1,20,11,17,"[0, 1, 0, 2, 2, 2, 2, 1, 2, 2, 2, 1, 1, 2, 1, ...","[6, 17]"
2,30,3,4,"[2, 2, 1, 2]","[1, 4]"
3,40,1,1,[2],"[0, 1]"
4,50,3,4,"[2, 2, 2, 2]","[1, 4]"
...,...,...,...,...,...
3677,521340,2,2,"[2, 2]","[0, 2]"
3678,521430,1,1,[2],"[0, 1]"
3679,521570,2,2,"[0, 0]","[0, 2]"
3680,521990,1,1,[2],"[0, 1]"


In [ ]:
df_para_steam_games.to_parquet('/content/drive/My Drive/DataOutput_mvp_steam/df_para_steam_games.parquet', index=False)

      Crear 'reviews_posted'

In [ ]:
'''
Creamos un dataframe 'reviews_posted' donde sus columnas serán 'user_id', 'posted', 'last_edited',
'recomendation' y 'comments', la idea es que la columna 'user_id' sera el id del usuario, luego
'posted' y 'last_posted' son las fechas de posteo y ultima edicion del posteo original y 'recomendation'
sera el valor true o false indicando si el usuario recomienda el juego o no, y por último, 'comments'
será la review como tal. La idea es que estamos por asi decirlo desanidando es por eso que la columna
'user_id' será la que tenga duplicados en función al tamaño de la lista de diccionarios
'''

user_ids = []
posted_dates = []
last_edited_dates = []
recomendations = []
comments = []

# Itera a través de las filas del DataFrame original
for _, row in user_reviews.iterrows():
    user_id = row['user_id']
    reviews = row['reviews']

    # Itera a través de las revisiones en la lista de diccionarios
    for review in reviews:
        posted_date = review.get('posted')
        last_edited_date = review.get('last_edited')
        recomendation = review.get('recommend')
        comentario = review.get('review')

        # Verifica si las fechas existen y no son None
        if (posted_date is not None or last_edited_date is not None):
            user_ids.append(user_id)
            posted_dates.append(posted_date)
            last_edited_dates.append(last_edited_date)
            recomendations.append(recomendation)
            comments.append(comentario)

# Crea un nuevo DataFrame con los datos extraídos
reviews_posted = pd.DataFrame({'user_id': user_ids,
                               'posted': posted_dates,
                               'last_edited': last_edited_dates,
                               'recomendation': recomendations,
                               'comment': comments})

In [ ]:
reviews_posted

,user_id,posted,last_edited,recomendation,comment
0,76561197970982479,2011/11/05,,True,2
1,76561197970982479,2011/07/15,,True,2
2,76561197970982479,2011/04/21,,True,2
3,js41637,2014/06/24,,True,2
4,js41637,2013/09/08,,True,2
...,...,...,...,...,...
58425,76561198312638244,2023/07/10,,True,2
58426,76561198312638244,2023/07/08,,True,2
58427,LydiaMorley,2023/07/03,,True,2
58428,LydiaMorley,2023/07/20,,True,2


In [ ]:
# Función para convertir fechas al formato "año/mes/día"
def convertir_fecha(fecha):
    try:
        # Intenta convertir la fecha utilizando dateutil.parser
        fecha_obj = dparser.parse(str(fecha), fuzzy=True)
        # Si la conversión es exitosa, formatea y devuelve la fecha en el nuevo formato
        return fecha_obj.strftime("%Y/%m/%d")
    except ValueError:
        # Si ocurre un error, simplemente devuelve la fecha original sin cambios
        return None

# Convertir 'posted' al formato "año/mes/día"
reviews_posted['posted'] = reviews_posted['posted'].apply(convertir_fecha)
# Convertir 'last_edited' al formato "año/mes/día"
reviews_posted['last_edited'] = reviews_posted['last_edited'].apply(convertir_fecha)

In [ ]:
# Convertir 'posted' y 'last_edited' al formato de fecha y hora
reviews_posted['posted'] = pd.to_datetime(reviews_posted['posted'])
reviews_posted['last_edited'] = pd.to_datetime(reviews_posted['last_edited'])

In [ ]:
reviews_posted = reviews_posted.reset_index(drop=True)

In [ ]:
reviews_posted

,user_id,posted,last_edited,recomendation,comment
0,76561197970982479,2011-11-05,NaT,True,2
1,76561197970982479,2011-07-15,NaT,True,2
2,76561197970982479,2011-04-21,NaT,True,2
3,js41637,2014-06-24,NaT,True,2
4,js41637,2013-09-08,NaT,True,2
...,...,...,...,...,...
58425,76561198312638244,2023-07-10,NaT,True,2
58426,76561198312638244,2023-07-08,NaT,True,2
58427,LydiaMorley,2023-07-03,NaT,True,2
58428,LydiaMorley,2023-07-20,NaT,True,2


In [ ]:
reviews_posted.to_parquet('/content/drive/My Drive/DataOutput_mvp_steam/reviews_posted.parquet')

    Crear la columna 'porcentaje_recomendacion'

In [ ]:
# Define una función para calcular el porcentaje de recomendación
def calcular_porcentaje_recomendacion(reviews):
    if not reviews:
        return None

    # Inicializar contadores
    count_true = 0
    count_total = 0

    # Iterar sobre las reviews en la lista de diccionarios
    for review in reviews:
        recommend = review.get('recommend')
        if recommend is not None:
            count_total += 1
            if recommend == True:
                count_true += 1

    # Calcular el porcentaje de recomendación
    if count_total > 0:
        porcentaje_recomendacion = (count_true / count_total) * 100
    else:
        porcentaje_recomendacion = None

    return porcentaje_recomendacion

# Aplicar la función a la columna 'reviews' y crear una nueva columna 'porcentaje_recomendacion'
user_reviews['porcentaje_recomendacion'] = user_reviews['reviews'].apply(calcular_porcentaje_recomendacion)

In [ ]:
user_reviews['reviews'][0]

[{'funny': '',
  'posted': '2011/11/05',
  'last_edited': '',
  'item_id': 1250,
  'helpful': 'No ratings yet',
  'recommend': True,
  'review': 2},
 {'funny': '',
  'posted': '2011/07/15',
  'last_edited': '',
  'item_id': 22200,
  'helpful': 'No ratings yet',
  'recommend': True,
  'review': 2},
 {'funny': '',
  'posted': '2011/04/21',
  'last_edited': '',
  'item_id': 43110,
  'helpful': 'No ratings yet',
  'recommend': True,
  'review': 2}]

In [ ]:
user_reviews['porcentaje_recomendacion']

0        100.0
1        100.0
2        100.0
3        100.0
4        100.0
         ...  
25480    100.0
25481    100.0
25482    100.0
25483    100.0
25484    100.0
Name: porcentaje_recomendacion, Length: 25485, dtype: float64

In [ ]:
# Función para verificar si al menos un diccionario en 'reviews' tiene 'recommend' como False
def contiene_false(reviews):
    for review in reviews:
        if 'recommend' in review and review['recommend'] is False:
            return True
    return False

# Aplicar la función para filtrar las filas
filtro = user_reviews['reviews'].apply(contiene_false)

# Filtrar el DataFrame
df_filtrado = user_reviews[filtro]

# Mostrar las filas filtradas
print(df_filtrado)

                 user_id                                           user_url  \
16     76561198043472122  http://steamcommunity.com/profiles/76561198043...   
24     76561198066046412  http://steamcommunity.com/profiles/76561198066...   
26     76561198070565427  http://steamcommunity.com/profiles/76561198070...   
36               boydeer               http://steamcommunity.com/id/boydeer   
52           sandwiches1           http://steamcommunity.com/id/sandwiches1   
...                  ...                                                ...   
25454        kimjongadam           http://steamcommunity.com/id/kimjongadam   
25458      iwishihadaids         http://steamcommunity.com/id/iwishihadaids   
25471       MeloncraftLP          http://steamcommunity.com/id/MeloncraftLP   
25472  76561198277602337  http://steamcommunity.com/profiles/76561198277...   
25473             943525                http://steamcommunity.com/id/943525   

                                                 re

In [ ]:
user_reviews

,user_id,user_url,reviews,porcentaje_recomendacion
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': '2011/11/05', 'last_e...",100.0
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': '2014/06/24', 'last_e...",100.0
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': '2023/02/03', 'last_e...",100.0
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': '2013/10/14', 'last_e...",100.0
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': 3, 'posted': '2014/04/15', 'last_ed...",100.0
...,...,...,...,...
25480,76561198306599751,http://steamcommunity.com/profiles/76561198306...,"[{'funny': '', 'posted': '2023/05/31', 'last_e...",100.0
25481,Ghoustik,http://steamcommunity.com/id/Ghoustik,"[{'funny': '', 'posted': '2023/06/17', 'last_e...",100.0
25482,76561198310819422,http://steamcommunity.com/profiles/76561198310...,"[{'funny': 1, 'posted': '2023/06/23', 'last_ed...",100.0
25483,76561198312638244,http://steamcommunity.com/profiles/76561198312...,"[{'funny': '', 'posted': '2023/07/21', 'last_e...",100.0


In [ ]:
user_reviews = user_reviews.drop(columns=['user_url', 'reviews'])

In [ ]:
user_reviews = user_reviews.reset_index(drop=True)

In [ ]:
user_reviews

,user_id,porcentaje_recomendacion
0,76561197970982479,100.0
1,js41637,100.0
2,evcentric,100.0
3,doctr,100.0
4,maplemage,100.0
...,...,...
25480,76561198306599751,100.0
25481,Ghoustik,100.0
25482,76561198310819422,100.0
25483,76561198312638244,100.0


In [ ]:
user_reviews.to_parquet('/content/drive/My Drive/DataOutput_mvp_steam/user_reviews.parquet')